# Pozostałe zasady

Cztery dodatkowe zasady GRASP: Controller, Pure Fabrication, Indirection, Protected Variations.

## 1. Controller

**Problem**: Klasy UI są z natury duże i złożone (wiele elementów, layoutów, stylów). Jeśli dodasz do nich logikę biznesową (co się dzieje po kliknięciu), klasa stanie się ogromna i nieczytelna.

**Pytanie**: Która klasa powinna odpowiadać za obsługę zdarzeń systemowych (np. kliknięcia użytkownika)?

**Odpowiedź**: Dedykowana klasa **Controller**, która:
- Odbiera zdarzenia z UI/API
- Deleguje zadania do odpowiednich klas biznesowych
- Nie zawiera logiki biznesowej (tylko koordynacja)

**Korzyść**: Separacja - UI zajmuje się wyglądem, Controller zachowaniem.

### Schemat

In [ ]:
# Warstwa UI
class UserInterface:
    def __init__(self, controller):
        self.controller = controller
    
    def on_button_click(self):
        # UI nie zawiera logiki, tylko przekazuje do controllera
        self.controller.handle_purchase()

# Warstwa kontrolera
class PurchaseController:
    def __init__(self, order_service, payment_service):
        self.order_service = order_service
        self.payment_service = payment_service
    
    def handle_purchase(self):
        # Kontroler koordynuje, nie implementuje logiki
        order = self.order_service.create_order()
        self.payment_service.process_payment(order)

# Warstwa serwisów (logika biznesowa)
class OrderService:
    def create_order(self):
        print("Tworzenie zamówienia")
        return "order_123"

class PaymentService:
    def process_payment(self, order):
        print(f"Przetwarzanie płatności dla {order}")

# Użycie
order_service = OrderService()
payment_service = PaymentService()
controller = PurchaseController(order_service, payment_service)
ui = UserInterface(controller)

ui.on_button_click()

**Kontroler nie robi nic oprócz koordynacji**. Logika znajduje się w `OrderService` i `PaymentService`.

### Związek z wzorcami
- **MVC (Model-View-Controller)** - Controller to środkowa warstwa
- **Facade** - Controller może pełnić rolę fasady dla podsystemu
- **Mediator** - Controller koordynuje komunikację między obiektami

**Korzyści**:
- Separacja UI od logiki biznesowej
- Łatwiejsze testowanie (można przetestować kontroler bez UI)
- Centralizacja obsługi zdarzeń

## 2. Pure Fabrication

Tworzenie bytów wyłącznie w celu rozluźnienia sprzęźeń i/lub zwiększenia spójności bez odzwierciedlenia zaproponowanego podziału w klasach projektowych.

**Problem**: Czasami przypisanie odpowiedzialności według Information Expert prowadzi do:
- Niskiej spójności (High Cohesion)
- Silnych sprzężeń (Low Coupling)
- Trudności w reużyciu kodu

**Rozwiązanie**: Stwórz **sztuczną klasę** (pure fabrication), która nie ma odpowiednika w rzeczywistości, ale rozwiązuje problem.

### Przykład: Zapis do bazy

**Źle** (Information Expert, ale niska spójność):

In [ ]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    # User "wie" o sobie, więc zapisuje siebie (Information Expert)
    def save_to_database(self):
        print(f"INSERT INTO users VALUES ('{self.name}', '{self.email}')")
    
    def load_from_database(self, user_id):
        print(f"SELECT * FROM users WHERE id={user_id}")

**Problem**:
- `User` ma 2 odpowiedzialności: dane + persystencja
- Ciężko reużyć logikę zapisu dla innych klas
- Zmiana bazy danych wymaga edycji `User`

**Dobrze** (Pure Fabrication):

In [2]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email

# Pure Fabrication - klasa "wymyślona", bez odpowiednika w rzeczywistości
class UserRepository:
    def save(self, user: User):
        print(f"INSERT INTO users VALUES ('{user.name}', '{user.email}')")
    
    def load(self, user_id: int) -> User:
        print(f"SELECT * FROM users WHERE id={user_id}")
        return User("John", "john@example.com")  # Symulacja


In [3]:
# Użycie
user = User("Alice", "alice@example.com")
repo = UserRepository()
repo.save(user)

INSERT INTO users VALUES ('Alice', 'alice@example.com')


**Korzyści**:
- `User` ma tylko 1 odpowiedzialność (dane)
- `UserRepository` można reużyć dla różnych klas
- Zmiana bazy = edycja tylko `UserRepository`

### Związek z SOLID
Pure Fabrication = **SRP** (Single Responsibility Principle) - wydzielamy odpowiedzialność do osobnej klasy.

## 3. Indirection

**Problem**: Bezpośrednie powiązanie między dwoma klasami prowadzi do silnego sprzężenia.

**Rozwiązanie**: Dodaj **pośrednika** (indirection) - klasę pośredniczącą między nimi.

### Przykład: Powiadomienia

**Źle** (bezpośrednie sprzężenie):

In [ ]:
class OrderService:
    def create_order(self):
        print("Zamówienie utworzone")
        # Bezpośrednie wywołanie EmailService
        EmailService().send("Zamówienie potwierdzone")

**Problem**: `OrderService` jest silnie powiązany z `EmailService`. Zmiana na SMS wymaga edycji `OrderService`.

**Dobrze** (Indirection):

In [ ]:
from abc import ABC, abstractmethod

# Pośrednik (interfejs)
class NotificationService(ABC):
    @abstractmethod
    def send(self, message: str):
        pass

class EmailNotification(NotificationService):
    def send(self, message: str):
        print(f"Email: {message}")

class SMSNotification(NotificationService):
    def send(self, message: str):
        print(f"SMS: {message}")

class OrderService:
    def __init__(self, notifier: NotificationService):
        self.notifier = notifier  # Zależność od interfejsu
    
    def create_order(self):
        print("Zamówienie utworzone")
        self.notifier.send("Zamówienie potwierdzone")

# Użycie
order_service = OrderService(EmailNotification())
order_service.create_order()

# Zmiana na SMS - zero edycji OrderService
order_service2 = OrderService(SMSNotification())
order_service2.create_order()

**Korzyści**:
- `OrderService` nie zależy od konkretnej implementacji
- Zmiana Email → SMS = zmiana argumentu, nie kodu
- Łatwiejsze testowanie (mock `NotificationService`)


## 4. Protected Variations

**Idea**: Wprowadź abstrakcję, która chroni jedną część systemu przed zmianami w drugiej części.

**Problem**: Zmiany w jednej części systemu powodują kaskadowe zmiany w innych.

**Rozwiązanie**: Chroń system przed zmianami poprzez **stabilny interfejs** (abstrakcję).

### Przykład: Eksport danych

**Źle** (brak ochrony przed zmianami):

In [ ]:
class DataExporter:
    def export(self, data, format):
        if format == "json":
            print(f"Eksportuję JSON: {data}")
        elif format == "xml":
            print(f"Eksportuję XML: {data}")
        # Nowy format = edycja if/elif

**Dobrze** (Protected Variations):

In [ ]:
class Exporter(ABC):
    @abstractmethod
    def export(self, data):
        pass

class JSONExporter(Exporter):
    def export(self, data):
        print(f"Eksportuję JSON: {data}")

class XMLExporter(Exporter):
    def export(self, data):
        print(f"Eksportuję XML: {data}")

class DataProcessor:
    def __init__(self, exporter: Exporter):
        self.exporter = exporter  # Chronione przez interfejs
    
    def process(self, data):
        # DataProcessor chroniony przed zmianami w formatach eksportu
        self.exporter.export(data)


In [ ]:
# Użycie
processor = DataProcessor(JSONExporter())
processor.process({"name": "Alice"})

# Nowy format CSV - zero zmian w DataProcessor
class CSVExporter(Exporter):
    def export(self, data):
        print(f"Eksportuję CSV: {data}")

processor2 = DataProcessor(CSVExporter())
processor2.process({"name": "Bob"})

**Korzyści**:
- `DataProcessor` **chroniony** przed zmianami w formatach eksportu
- Nowy format = nowa klasa, nie edycja `DataProcessor`
- Stabilny interfejs (`Exporter`) izoluje zmiany

**Kluczowa obserwacja**: Abstrakcja (`Exporter`) pozwala podmienić implementację bez wpływu na kod używający. Możesz dodawać/usuwać formaty eksportu, a `DataProcessor` pozostaje niezmieniony.

### Związek z SOLID
Protected Variations = **OCP** (Open/Closed Principle) - zamknięty na modyfikacje, otwarty na rozszerzenia.

### Techniki ochrony przed zmianami
1. **Interfejsy/klasy abstrakcyjne** - stabilny kontrakt
2. **Polimorfizm** - różne implementacje tego samego interfejsu
3. **Enkapsulacja** - ukrycie szczegółów implementacji
4. **Dependency Injection** - zależności wstrzykiwane z zewnątrz

### Kluczowa obserwacja
Większość zasad GRASP to **konkretne zastosowania zasad SOLID**:
- Pure Fabrication = SRP
- Indirection = DIP
- Protected Variations = OCP
- Polymorphism = LSP + OCP

**GRASP** (1997) to praktyczne zastosowania zasad, które później zostały sformalizowane jako **SOLID** (2000).